In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Lambda, Conv2DTranspose, SeparableConv2D
from dataset import create_artifact_dataset
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
"""from tensorflow.keras.backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)"""

'from tensorflow.keras.backend import set_session\nimport tensorflow as tf\nconfig = tf.ConfigProto()\nconfig.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU\nconfig.log_device_placement = True  # to log device placement (on which device the operation ran)\nsess = tf.Session(config=config)\nset_session(sess)'

In [3]:
def get_ARCNN(input_shape=(32,32,1)):
    inp = Input(shape=input_shape)
    conv1 = Conv2D(64,9,activation='relu', padding='same', use_bias=True,name="Feature_extract")(inp)
    conv2 = Conv2D(32,7,activation='relu', padding='same', use_bias=True,name="Feature_Enhance")(conv1)
    conv3 = Conv2D(64,1,activation='relu', padding='valid', use_bias=True,name="Mapping")(conv2)
    conv_trans = Conv2DTranspose(1,7,padding='same')(conv3)
    ARCNN = Model(inputs=inp,outputs=conv_trans,name="ARCNN")
    return ARCNN

def get_Fast_ARCNN(input_shape=(32,32,1)):
    inp = Input(shape=input_shape)
    conv1 = Conv2D(64,9,activation='relu', padding='same', use_bias=True,name="Feature_extract")(inp)
    conv2 = Conv2D(32,1,activation='relu', padding='valid', use_bias=True,name="Feature_Enhance_speed")(conv1)
    conv3 = Conv2D(32,7,activation='relu', padding='same', use_bias=True,name="Feature_Enhance")(conv2)
    conv4 = Conv2D(64,1,activation='relu', padding='valid', use_bias=True,name="Mapping")(conv3)
    conv_trans = Conv2DTranspose(1,7,padding='same')(conv4)
    ARCNN = Model(inputs=inp,outputs=conv_trans,name="Faster_ARCNN")
    return ARCNN

def get_ARCNN_lite(input_shape=(32,32,1)):
    inp = Input(shape=input_shape)
    conv1 = Conv2D(32,5,dilation_rate=4,activation='relu', padding='same', use_bias=True,name="Feature_extract")(inp)
    conv2 = Conv2D(32,1,activation='relu', padding='valid', use_bias=True,name="Feature_Enhance_speed")(conv1)
    conv3 = Conv2D(32,5,dilation_rate=2,activation='relu', padding='same', use_bias=True,name="Feature_Enhance")(conv2)
    conv4 = Conv2D(32,1,activation='relu', padding='valid', use_bias=True,name="Mapping")(conv3)
    conv_trans = Conv2DTranspose(1,3,dilation_rate=4,name="Upscale",padding='same')(conv4)
    ARCNN = Model(inputs=inp,outputs=conv_trans)
    return ARCNN

In [4]:
ARCNN = get_ARCNN()
ARCNN.summary()

Model: "ARCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
Feature_extract (Conv2D)     (None, 32, 32, 64)        5248      
_________________________________________________________________
Feature_Enhance (Conv2D)     (None, 32, 32, 32)        100384    
_________________________________________________________________
Mapping (Conv2D)             (None, 32, 32, 64)        2112      
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 1)         3137      
Total params: 110,881
Trainable params: 110,881
Non-trainable params: 0
_________________________________________________________________


In [5]:
Faster_ARCNN = get_Fast_ARCNN()
Faster_ARCNN.summary()

Model: "Faster_ARCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
Feature_extract (Conv2D)     (None, 32, 32, 64)        5248      
_________________________________________________________________
Feature_Enhance_speed (Conv2 (None, 32, 32, 32)        2080      
_________________________________________________________________
Feature_Enhance (Conv2D)     (None, 32, 32, 32)        50208     
_________________________________________________________________
Mapping (Conv2D)             (None, 32, 32, 64)        2112      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 1)         3137      
Total params: 62,785
Trainable params: 62,785
Non-trainable params: 0
__________________________________________________

In [6]:
ARCNN_lite = get_ARCNN_lite()
ARCNN_lite.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
Feature_extract (Conv2D)     (None, 32, 32, 32)        832       
_________________________________________________________________
Feature_Enhance_speed (Conv2 (None, 32, 32, 32)        1056      
_________________________________________________________________
Feature_Enhance (Conv2D)     (None, 32, 32, 32)        25632     
_________________________________________________________________
Mapping (Conv2D)             (None, 32, 32, 32)        1056      
_________________________________________________________________
Upscale (Conv2DTranspose)    (None, 32, 32, 1)         289       
Total params: 28,865
Trainable params: 28,865
Non-trainable params: 0
_________________________________________________________

In [8]:
data = create_artifact_dataset()
data = data.prefetch(tf.data.experimental.AUTOTUNE)

flist: 823
Shapes
Tensor("strided_slice:0", shape=(), dtype=int32)
y_patches : (None, 100, 100, 1)
x_patches : (None, 100, 100, 1)


In [1]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
ARCNN_v1.compile(optimizer=optimizer,loss=tf.keras.losses.MeanSquaredError(),metrics=[ssim,psnr])

NameError: name 'tf' is not defined

In [54]:
tboard = tf.keras.callbacks.TensorBoard(log_dir="./logs/no_batch",write_images=True)
es = tf.keras.callbacks.EarlyStoppinag()

In [55]:
ARCNN_v1.fit(data,
             epochs=2,
             callbacks=[tboard,es])

Epoch 1/2
   3292/Unknown - 115s 35ms/step - loss: 0.0022 - ssim: 0.8762 - psnr: 30.9640WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,ssim,psnr
3292/3292 [==============================] - 115s 35ms/step - loss: 0.0022 - ssim: 0.8762 - psnr: 30.9640
Epoch 2/2
3292/3292 [==============================] - 116s 35ms/step - loss: 0.0014 - ssim: 0.8851 - psnr: 31.7316
